# Python REPL

Sometimes, for complex calculations, rather than have an LLM generate the answer directly, it can be better to have the LLM generate code to calculate the answer, and then run that code to get the answer. In order to easily do that, we provide a simple Python REPL to execute commands in.

This interface will only return things that are printed - therefore, if you want to use it to calculate an answer, make sure to have it print out the answer.


:::{.callout-caution}
Python REPL can execute arbitrary code on the host machine (e.g., delete files, make network requests). Use with caution.

For more information general security guidelines, please see https://python.langchain.com/v0.2/docs/security/.
:::

In [2]:
%pip install -q langchain_experimental

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL

In [4]:
python_repl = PythonREPL()

In [5]:
python_repl.run("print(1+1)")

Python REPL can execute arbitrary code. Use with caution.


'2\n'

In [6]:
# You can create the tool to pass to an agent
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)

## Example Usage with LLM

### Setting Up environment

In [8]:
import getpass
import os

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

(Recommended) Setting up LangSmith Trace

In [9]:
_set_env("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

### Creating an Agent with Python REPL tool

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

model = ChatOpenAI(model="gpt-4")

Now, let's bind the python REPL tool defined above to the llm

In [15]:
from langgraph.prebuilt import create_react_agent

tools = [repl_tool]

agent_executor = create_react_agent(model, tools)

Now let's try to solve an SAT math exam problem with our agent

In [22]:
system_instruction = """
You are a helpful assistant who performs accurate mathematical calculators. When asked a question, first reason through the question step by step, then follow the reasoning to solve the question. When you need to perform a mathematical operation, call the provided tools. """

question = """
If f(x) = 2x 2 + 4 for all real numbers x , what is the value of f(3) + f(5)?
"""

response = agent_executor.invoke(
    {"messages": [SystemMessage(content=system_instruction),
                  HumanMessage(content=question)]}
)
response["messages"][-1].content

[SystemMessage(content='\nYou are a helpful assistant who performs accurate mathematical calculators. When asked a question, first reason through the question step by step, then follow the reasoning to solve the question. When you need to perform a mathematical operation, call the provided tools. ', id='01490f4f-494c-4f67-a423-b177db3458c8'),
 HumanMessage(content='\nIf f(x) = 2x 2 + 4 for all real numbers x , what is the value of f(3) + f(5)?\n', id='04f1e4ec-4010-48b4-86d7-d17e4bf8aa28'),
 AIMessage(content="To solve this, we need to substitute x = 3 and x = 5 into the function f(x) = 2x^2 + 4, compute the values, and then add them together.\n\nLet's start by calculating f(3). We substitute x = 3 into the function, which gives us 2*3^2 + 4.\n\nThen we'll calculate f(5) by substituting x = 5 into the function, giving us 2*5^2 + 4.\n\nFinally, we add these two results together to get the final answer. \n\nLet's perform these calculations now.", additional_kwargs={'tool_calls': [{'id': 

The complete reasoning step:

In [30]:
from langchain_core.output_parsers import StrOutputParser
for message in response["messages"]:
    print(StrOutputParser().invoke(message))
    print('-'*80)


You are a helpful assistant who performs accurate mathematical calculators. When asked a question, first reason through the question step by step, then follow the reasoning to solve the question. When you need to perform a mathematical operation, call the provided tools. 
--------------------------------------------------------------------------------

If f(x) = 2x 2 + 4 for all real numbers x , what is the value of f(3) + f(5)?

--------------------------------------------------------------------------------
To solve this, we need to substitute x = 3 and x = 5 into the function f(x) = 2x^2 + 4, compute the values, and then add them together.

Let's start by calculating f(3). We substitute x = 3 into the function, which gives us 2*3^2 + 4.

Then we'll calculate f(5) by substituting x = 5 into the function, giving us 2*5^2 + 4.

Finally, we add these two results together to get the final answer. 

Let's perform these calculations now.
---------------------------------------------------